In [1]:
#!pip install sympy

In [1]:
from sympy import *
# init_session()
from __future__ import division
x, y, z, t = symbols('x y z t')
k, m, n = symbols('k m n', integer=True)
f, g, h = symbols('f g h', cls=Function)
init_printing(use_latex=False)

from IPython.display import Markdown as md
from IPython.display import display


In [2]:
from sympy import *
from IPython.display import display_latex, display
init_session()


def sistema_a_latex(sist) :
    """
    Convierte un sistema Sympy a código LaTeX
    """
    sist_latex = r"$\left\{ \begin{matrix}"
    for e in sist:
        sist_latex += latex(e)+r" \\ "  
    sist_latex += r"\end{matrix}\right.$"
    return(sist_latex)


def enunciado_dicusion(sist, k=k,lista=[]) :
    """
    Típico enunciado de discusión y resolución de un sistema con un parámetro
    """
    sistema_problema = sist
    texto_problema = r"""Considere el siguiente sistema de ecuaciones, dónde {}  es un parámetro real: $""".format(k)
    texto_problema += sistema_a_latex(sistema_problema)
    texto_problema += """$ Determine los valores del parámetro real {}, para los que este sistema es compatible determinado,  compatible indeterminado o incompatible.""".format(k)
    
    if lista :
        texto_problema +=r" \\ Resuelva el sistema cuando"
        texto_problema +=r"\begin{itemize}"
        for i in lista :
            texto_problema += r"\item {}={}".format(k,i)        
        texto_problema +=r"\end{itemize}"
    return(texto_problema)


def discusion_solucion(sist, k=k, resol=True) :
    """
    Discute y resuelve un sistema con un parámetro o sin él
    """
    d=dict()
    solucion_md=[]
    A, b = linear_eq_to_matrix(sist,[x,y,z])
    AA = A.row_join(b)
    AAs = AA.LUdecomposition()[1].applyfunc(nsimplify)
    
    if A.rank() < AA.rank():
        solucion_latex = r"Como $rg(A)={} < rg(A^*)={} \to$ S.I. \\ ".format(A.rank(),AA.rank())
        solucion_latex += r"Ya que escalonando la matriz ampliada: \\ $A^*= {} \thicksim {}$ \\".format(latex(AA),latex(AAs)).replace('[','(').replace(']',')')
        solucion_latex += r"${}={} \quad y \quad {}={}$".format(latex(AAs[:,:-1]),latex(A.det()),latex(AAs[:,1:]),latex(AAs[:,1:].det())).replace('[','|').replace(']','|')
        solucion_md.append(r"Como $rg(A)={} < rg(A^*)={} \to$ S.I. \\ ".format(A.rank(),AA.rank()))
        solucion_md.append(r"Ya que escalonando la matriz ampliada: \\ $A^*= {} \thicksim {}$ \\".format(latex(AA),latex(AAs)).replace('[','(').replace(']',')'))
        solucion_md.append(r"${}={} \quad y \quad {}={}$".format(latex(AAs[:,:-1]),latex(A.det()),latex(AAs[:,1:]),latex(AAs[:,1:].det())).replace('[','|').replace(']','|'))

        
    else :  
        pprint("Discusión y resolución por Gauss:")
        solucion_md.append("**Discusión y resolución por Gauss** ")
        solucion_md.append("Escalonando la matriz ampliada tenemos:")
        solucion_md.append(r"$A = {} \thicksim {}= A^*$.".format(latex(AA),latex(AAs)).replace('[','(').replace(']',')'))
        solucion_md.append("De los valores de la última fila podemos concluir:")
 
        solucion_latex = r"\textbf{Discusión y resolución por Gauss:} Escalonando la matriz ampliada tenemos\\"
        solucion_latex += r"$A^*= {} \thicksim {}$. \\  De los valores de la última fila podemos concluir:".format(latex(AA),latex(AAs)).replace('[','(').replace(']',')')
        solucion_latex += r"\begin{itemize}"
        pprint(AAs)
                
        pprint("det(A)={}".format(A.det()))
        pprint(AAs.row(-1)[-2:])

        for i in (solve(AAs.row(-1)[-2])) :
            pprint(i)
            if AAs.row(-1)[-1].subs(k,i) == 0 :
                pprint("Si {} = {} -->  0z=0 --> S.C.I".format(k,i))
                solucion_md.append(r"* Si ${} = {} \to$ $${}$$ La última fila es $0z=0 \to $ S.C.I".format(k,i,latex(AAs.subs(k,i))).replace('[','(').replace(']',')'))
                solucion_latex += r"\item Si ${} = {} \to$ $${}$$ La última fila es $0z=0 \to $ S.C.I".format(k,i,latex(AAs.subs(k,i))).replace('[','(').replace(']',')')
                pprint([eq.subs(k,i) for eq in sist])
                sol = list(zip([x,y,z],linsolve([eq.subs(k,i) for eq in sist],[x,y,z]).args[0],[r.subs(k,i) for r in [AAs.row(j) for j in range(AA.shape[0])]]))
                pprint(sol)
                for s in reversed(sol) :
                        pprint("{} --> {} = {}".format(s[2],s[0],s[1].subs(z,"\lambda")))
                        if resol :
                            solucion_latex += r"\begin{itemize}"
                            solucion_md.append(r"    - ${} \to {} = {}$".format(latex(s[2]),latex(s[0]),latex(s[1]).replace('z',"\lambda")).replace('[','(').replace(']',')'))
                            solucion_latex += r"\item ${} \to {} = {}$".format(latex(s[2]),latex(s[0]),latex(s[1]).replace('z',"\lambda")).replace('[','(').replace(']',')')
                            solucion_latex += r"\end{itemize}"
            else :
                pprint("Si {} = {} --> 0z={} -->S.I".format(k,i, AAs.row(-1)[-1].subs(k,i)))
                solucion_latex += r"\item Si ${} = {} \to$ $${}$$ La última fila es $0z={} \to $ S.I.".format(k,i,latex(AAs.subs(k,i)),AA.LUdecomposition()[1].applyfunc(simplify).row(-1)[-1].subs(k,i)).replace('[','(').replace(']',')')
                solucion_md.append(r"* Si ${} = {} \to$ $${}$$ La última fila es $0z={} \to $ S.I.".format(k,i,latex(AAs.subs(k,i)),AA.LUdecomposition()[1].applyfunc(simplify).row(-1)[-1].subs(k,i)).replace('[','(').replace(']',')'))

        if solve(AAs.row(-1)[-2]) :
            pprint("si {} <> {}  --> S.C.D.".format(k, solve(AAs.row(-1)[-2])))
            solucion_latex += r"\item si ${}\neq {}  \to $ S.C.D.".format(k,solve(AAs.row(-1)[-2]))
            solucion_md.append(r"* Si ${}\neq {} $ S.C.D.".format(k,solve(AAs.row(-1)[-2])))

        else :
            pprint("S.C.D.".format(k, solve(AAs.row(-1)[-2])))
            solucion_latex += r"\item S.C.D.".format(k,solve(AAs.row(-1)[-2]))
            solucion_md.append(r"S.C.D.")
            


        pprint(list(linsolve(sist,[x,y,z]).args[0].args))
        sol = list(zip([x,y,z],linsolve(sist,[x,y,z]).args[0],[AAs.row(j) for j in range(AA.shape[0])]))
        pprint(sol)
        if resol :
            for s in reversed(sol) :
                pprint("{} --> {} = {}".format(s[2],s[0],s[1]))
                solucion_latex += r"\begin{itemize}"
                solucion_latex += r"\item ${} \to {} = {}$".format(latex(s[2]),s[0],latex(s[1])).replace('[','(').replace(']',')')
                solucion_md.append(" * ${} \\to {} = {}$".format(latex(s[2]),s[0],latex(s[1])).replace('[','(').replace(']',')'))
                solucion_latex += r"\end{itemize}"

        solucion_latex += r"\end{itemize}  "
        
        # Por rangos y determinantes

        pprint("Por rangos y determinantes:")
        solucion_latex += r"\textbf{Por rangos y determinantes:} \\"
        solucion_md.append(r"**Por rangos y determinantes**")


        #Rango de A:


        solucion_latex += r"$\left|A\right|={}={} ".format(latex(A).replace("[","|").replace("]","|"),latex(A.det()))
        txt_md= r"$\left|A\right|={}={} ".format(latex(A).replace("[","|").replace("]","|"),latex(A.det()))
        
        if solve(A.det()) :
            solucion_latex += r"\to \left|A\right|=0 \quad si \quad {} = {}$".format(k,latex(solve(A.det())))
            txt_md += r"\to \left|A\right|=0 \quad si \quad {} = {}$".format(k,latex(solve(A.det())))
            solucion_latex += r"\begin{itemize}"
            for i in solve(A.det()):
                A2 = A.subs(k,i)
                AA2 = AA.subs(k,i)
                pprint(A2)
                pprint(A2.rank())
                pprint(AA2.rank())
                if A2.rank() < AA2.rank() :
                    pprint("Si {} = {} --> S.I.".format(k,i))
                    solucion_latex += r"\item Si ${}={} \to rg(A)={} \land rg(A^*)={} \to $ S.I.".format(k,i,A2.rank(),AA2.rank())
                    txt_md += r"* Si ${}={} \to rg(A)={} \land rg(A^*)={} \to $ S.I.".format(k,i,A2.rank(),AA2.rank())
                else :
                    pprint("Si {} = {} --> S.C.I --> solo se puede resolver por Gauss, ver más arriba".format(k,i))
                    solucion_latex += r"\item Si ${}={} \to rg(A)={} \land rg(A^*)={} \to $ S.C.I. $\to$ solo se puede resolver por Gauss, (ver más arriba)".format(k,i,A2.rank(),AA2.rank())
                    txt_md += r" * Si ${}={} \to rg(A)={} \land rg(A^*)={} \to $ S.C.I. $\to$ solo se puede resolver por Gauss, (ver más arriba)".format(k,i,A2.rank(),AA2.rank())
            pprint("Si {} <> {} --> Rango(A)=Rango(A)={} --> S.C.D.".format(k,solve(A.det()),A.rank()))
            solucion_latex += r"\item Si ${} \neq{} \to rg(A)={} \land rg(A^*)={} \to $ S.C.D.  \\ ".format(k, solve(A.det()),A.rank(),AA.rank())
            txt_md += r"  \n * Si ${} \neq{} \to rg(A)={} \land rg(A^*)={} \to $ S.C.D.  \\ ".format(k, solve(A.det()),A.rank(),AA.rank())
            solucion_md.append(txt_md)

            if resol :
                pprint("Por Cramer:")
                sol=[]
                solucion_latex += r" \\ Por Cramer: \begin{itemize}"
                for i, var in enumerate([x,y,z]):
                    #print("columna"+latex(i))
                    AA.col_swap(i,3)
                    pprint("Delta_i, A.det, solucion_latex_i : ")
                    sol.append([var,AA[:,:-1],simplify(AA[:,:-1].det()),simplify(A.det()),simplify(AA[:,:-1].det()/A.det())])
                    pprint(sol[i])
                    #print(r"$x_"+latex(i)+r"=\frac{"+latex(AA[:,:-1])+r"}{"+latex(A.det())+r"}=\frac{"+latex(AA[:,:-1].det())+r"}{"+latex(A.det())+r"}="+latex(AA[:,:-1].det()/A.det())+"$")
                    solucion_latex += r"\item $"+latex(sol[i][0])+r"=\frac{"+latex(sol[i][1]).replace('[','|').replace(']','|')+r"}{"+latex(sol[i][3])+r"}=\frac{"+latex(sol[i][2])+r"}{"+latex(sol[i][3])+r"}="+latex(sol[i][4])+r"$"
                    AA.col_swap(i,3)
                solucion_latex += r"\end{itemize}"

        else:
            solucion_latex += r" \neq 0 $"
            txt_md += r" \neq 0 $"
            solucion_latex += r"\begin{itemize}"
            solucion_latex += r" \item $rg(A)={} \land rg(A^*)={} \to $ S.C.D.   \\".format(A.rank(),AA.rank())
            txt_md += "  \n * $rg(A)={} \land rg(A^*)={} \\to $ S.C.D.".format(A.rank(),AA.rank())
            if resol :
                pprint("Por Cramer:")
                sol=[]
                solucion_latex += r" \\ Por Cramer: \begin{itemize}"
                txt_md += "\n"+r"**Por Cramer**"
                for i, var in enumerate([x,y,z]):
                    #print("columna"+latex(i))
                    AA.col_swap(i,3)
                    pprint("Delta_i, A.det, solucion_latex_i : ")
                    sol.append([var,AA[:,:-1],simplify(AA[:,:-1].det()),simplify(A.det()),simplify(AA[:,:-1].det()/A.det())])
                    pprint(sol[i])
                    #print(r"$x_"+latex(i)+r"=\frac{"+latex(AA[:,:-1])+r"}{"+latex(A.det())+r"}=\frac{"+latex(AA[:,:-1].det())+r"}{"+latex(A.det())+r"}="+latex(AA[:,:-1].det()/A.det())+"$")
                    solucion_latex += r"\item $"+latex(sol[i][0])+r"=\frac{"+latex(sol[i][1]).replace('[','|').replace(']','|')+r"}{"+latex(sol[i][3])+r"}=\frac{"+latex(sol[i][2])+r"}{"+latex(sol[i][3])+r"}="+latex(sol[i][4])+r"$"
                    txt_md += "   \n * "+"$"+latex(sol[i][0])+r"=\frac{"+latex(sol[i][1]).replace('[','|').replace(']','|')+r"}{"+latex(sol[i][3])+r"}=\frac{"+latex(sol[i][2])+r"}{"+latex(sol[i][3])+r"}="+latex(sol[i][4])+r"$"

                    AA.col_swap(i,3)
                solucion_latex += r"\end{itemize}"
        solucion_latex += r"\end{itemize}"
        solucion_md.append(txt_md)
    d['solucion_latex']=solucion_latex
    d['solucion_markdown']=solucion_md
    return(d) 



 

IPython console for SymPy 1.10.1 (Python 3.9.13-64-bit) (ground types: gmpy)

These commands were executed:
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.10.1/



In [3]:
"""
nueva versión para resolver sistemas v2223
"""

def lista_incognitas(sist, param=k) :
    '''
    extrae del sistema el listado de variables (letras) y no tiene en cuenta el parámetro si se le pasa
    '''
    l=[]
    for expr in sist:
        for s in expr.free_symbols:
            l.append(s)
    l=list(set(l))
    if param in l :
        l.remove(param)
    
    # ordenamos la lista
    l=list(map(str,l))
    l.sort(reverse=False)
    l=list(map(S,l))
    return(l)

def sistema_solucion(sist, l_incognitas) :
    """
    pasamos el sistema y el listado de incógnitas y se discute y resuelve
    """

    gauss, minversa, cramer = [False, False, False]
    l_tex=[]

    A, b = linear_eq_to_matrix(sist,l_incognitas)
    AA = A.row_join(b)
    AAs = AA.LUdecomposition()[1].applyfunc(nsimplify).applyfunc(simplify)

    
    # Discusión
    if A.rank() == AA.rank() :
        display_latex(r"Como $rg(A)=rg(A^*)={}$ --> S.C. (El sistema tiene solución)".format(A.rank()),raw=True)
        if A.rank() == len(l_incognitas) :
            display_latex(r"Como además $rg(A)={}$ coincide con el número de incógnitas --> S.C.D -- > Se puede resolver por Gauss, Matriz inversa o Cramer".format(A.rank()),raw=True)
            gauss = True
            minversa = True
            cramer = True
        else:
            display_latex(r"Como además $rg(A)={}$ no coincide con el número de incógnitas--> S.C.I --> Solo se puede resolver por Gauss".format(A.rank()),raw=True)
            gauss = True
    else :
        tex = r"Como $rg(A)={} \land rg(A^*)={}$ --> S.I. ".format(A.rank(),AA.rank())
        tex += ". Observa que escalonando la matriz ampliada obtenemos "
        tex += "$A^*="+latex(AA).replace("[","(").replace("]",")")+r"\sim"+latex(AAs).replace("[","(").replace("]",")")+r"$"
        display_latex(tex,raw=True)

    if gauss == True :
        print("**Resolución por Gauss**")
        print("- - - - - - - - - - - -")
        sol=linsolve(sist,l_incognitas).args[0].subs(z,t)
#         tex = "Escalonando la matriz ampliada obtenemos "
        tex = "$A^*="+latex(AA).replace("[","(").replace("]",")")+r"\sim"+latex(AAs).replace("[","(").replace("]",")")
        tex += r"\to "+", ".join([latex(s[0])+"="+latex(s[1]) for s in list(zip(l_incognitas,sol))]).replace('t',r'\lambda')
        tex +="$"
        display_latex(tex,raw=True)
     
    if minversa == True :
        print("**Método de la matríz inversa**")
        print("- - - - - - - - - - - - - - - ")
        tex =r"$A^{-1}="+latex(A.inv()).replace("[","(").replace("]",")")+r" \to "
        tex += r" X=A^{-1}\cdot b"
        tex += r" ="+latex(A.inv()).replace("[","(").replace("]",")")+r"\cdot "+latex(b).replace("[","(").replace("]",")")
        tex += r" ="+latex((A.inv()*b).applyfunc(simplify)).replace("[","(").replace("]",")")+r"$"      
        display_latex(tex,raw=True)
    if cramer == True :
        sol=[]
        print("**Por Cramer**")
        print("- - - - - - - -")
        for i, var in enumerate(l_incognitas):
            AA.col_swap(i,3)
            sol.append([var,AA[:,:-1],nsimplify(AA[:,:-1].det()),simplify(A.det()),simplify(AA[:,:-1].det())/A.det()])
            tex = r"$"+latex(sol[i][0])+r"=\frac{"+latex(sol[i][1]).replace('[','|').replace(']','|')+r"}{"+latex(sol[i][3])+r"}=\frac{"+latex(sol[i][2])+r"}{"+latex(sol[i][3])+r"}="+latex(sol[i][4])+r"$"
            display_latex(tex,raw=True)
            AA.col_swap(i,3)
       

        
        
        
        
        
    
def sistema_parametro(sist, param=k) :
    """
    Discute los diferentes sistemas que aparecen según el parámetro 
    """
    d=dict()
    solucion_md, solucion_latex = [], []
    
  
    
    l_incognitas = lista_incognitas(sist, param)   
#     print(l_incognitas)
#     print(param)
    n_incognitas = len(l_incognitas)
    
    
    
    A, b = linear_eq_to_matrix(sist,l_incognitas)
    AA = A.row_join(b)
    AAs = AA.LUdecomposition()[1].applyfunc(nsimplify).applyfunc(simplify)
    
    
    print("Discusión y resolución del sistema:")
    display_latex(sistema_a_latex(sist),raw=True)
    print("-----------------------------------------------------------")

    # Valores donde existe la inversa:
    tex = "$|A|="+latex(A).replace("[","|").replace("]","|")+"="+latex(factor(A.det()))+"$"
    display_latex(tex,raw=True)
    tex = r"* Si $"+latex(param)+r" \neq "+", ".join(list(map(str,solve(A.det()))))+r"\to |A| \neq 0 \to \exists A^{-1}$."
    display_latex(tex,raw=True)  
    sistema_solucion(sist,l_incognitas)
        
    # Donde no hay inversa    
    for i in solve(A.det()):
        display_latex(r"* Si $"+str(param)+r"="+str(i)+r"\to |A| = 0 \to \nexists A^{-1}$",raw=True)
        sistema_solucion(S(str(sist).replace(str(param),str(i))),l_incognitas)
        
  
    
    d['solucion_latex']=solucion_latex
    d['solucion_markdown']=solucion_md
    
    return(d)




In [5]:
# p14.e1

sistemas = [
            [Eq(x-2*y+3*z,-3),Eq(2*x+y,2), Eq(-x-8*y+9*z,3)],
            [Eq(x-2*y+z,0), Eq(x+y-2*z,-3),Eq(-2*x+5*y-z,5), Eq(3*x-2*y+z,2)],
            [Eq(x-y+2*z+3*t,1), Eq(2*x+3*y-z+t,-3),Eq(x+y+z+t,0)],
            [Eq(x-y+2*z,1-3*t), Eq(2*x+3*y-z,-3-t),Eq(x+y+z,-t)]
           ]

for sist in sistemas:
  pprint(sist)
  display_latex("$"+sistema_a_latex(sist)+"$",raw=True)
  display("$"+sistema_a_latex(sist)+"$")  
  A, b = linear_eq_to_matrix(sist,[x,y,z])
  AA = A.row_join(b)
  #pprint(linear_eq_to_matrix(sist,[x,y,z]))
  pprint([A,A.rank(),AA,AA.rank(),  AA.LUdecomposition()[1],A.shape])
  #pprint([A.LUdecomposition(),A.LUsolve(b)])
  pprint(linsolve(sist,[x,y,z]))
  pprint(solve(sist))

[x - 2⋅y + 3⋅z = -3, 2⋅x + y = 2, -x - 8⋅y + 9⋅z = 3]


'$$\\left\\{ \\begin{matrix}x - 2 y + 3 z = -3 \\\\ 2 x + y = 2 \\\\ - x - 8 y + 9 z = 3 \\\\ \\end{matrix}\\right.$$'

⎡⎡1   -2  3⎤     ⎡1   -2  3  -3⎤     ⎡1  -2  3   -3⎤        ⎤
⎢⎢         ⎥     ⎢             ⎥     ⎢             ⎥        ⎥
⎢⎢2   1   0⎥, 2, ⎢2   1   0  2 ⎥, 3, ⎢0  5   -6  8 ⎥, (3, 3)⎥
⎢⎢         ⎥     ⎢             ⎥     ⎢             ⎥        ⎥
⎣⎣-1  -8  9⎦     ⎣-1  -8  9  3 ⎦     ⎣0  0   0   16⎦        ⎦
∅
[]
[x - 2⋅y + z = 0, x + y - 2⋅z = -3, -2⋅x + 5⋅y - z = 5, 3⋅x - 2⋅y + z = 2]


'$$\\left\\{ \\begin{matrix}x - 2 y + z = 0 \\\\ x + y - 2 z = -3 \\\\ - 2 x + 5 y - z = 5 \\\\ 3 x - 2 y + z = 2 \\\\ \\end{matrix}\\right.$$'

⎡⎡1   -2  1 ⎤     ⎡1   -2  1   0 ⎤     ⎡1  -2  1   0 ⎤        ⎤
⎢⎢          ⎥     ⎢              ⎥     ⎢             ⎥        ⎥
⎢⎢1   1   -2⎥     ⎢1   1   -2  -3⎥     ⎢0  3   -3  -3⎥        ⎥
⎢⎢          ⎥, 3, ⎢              ⎥, 3, ⎢             ⎥, (4, 3)⎥
⎢⎢-2  5   -1⎥     ⎢-2  5   -1  5 ⎥     ⎢0  0   2   6 ⎥        ⎥
⎢⎢          ⎥     ⎢              ⎥     ⎢             ⎥        ⎥
⎣⎣3   -2  1 ⎦     ⎣3   -2  1   2 ⎦     ⎣0  0   0   0 ⎦        ⎦
{(1, 2, 3)}
{x: 1, y: 2, z: 3}
[3⋅t + x - y + 2⋅z = 1, t + 2⋅x + 3⋅y - z = -3, t + x + y + z = 0]


'$$\\left\\{ \\begin{matrix}3 t + x - y + 2 z = 1 \\\\ t + 2 x + 3 y - z = -3 \\\\ t + x + y + z = 0 \\\\ \\end{matrix}\\right.$$'

⎡⎡1  -1  2 ⎤     ⎡1  -1  2   1 - 3⋅t⎤     ⎡1  -1  2   1 - 3⋅t⎤        ⎤
⎢⎢         ⎥     ⎢                  ⎥     ⎢                  ⎥        ⎥
⎢⎢2  3   -1⎥, 3, ⎢2  3   -1  -t - 3 ⎥, 3, ⎢0  5   -5  5⋅t - 5⎥, (3, 3)⎥
⎢⎢         ⎥     ⎢                  ⎥     ⎢                  ⎥        ⎥
⎣⎣1  1   1 ⎦     ⎣1  1   1     -t   ⎦     ⎣0  0   1      1   ⎦        ⎦
{(-2⋅t - 1, t, 1)}
{t: y, x: -2⋅y - 1, z: 1}
[x - y + 2⋅z = 1 - 3⋅t, 2⋅x + 3⋅y - z = -t - 3, x + y + z = -t]


'$$\\left\\{ \\begin{matrix}x - y + 2 z = 1 - 3 t \\\\ 2 x + 3 y - z = - t - 3 \\\\ x + y + z = - t \\\\ \\end{matrix}\\right.$$'

⎡⎡1  -1  2 ⎤     ⎡1  -1  2   1 - 3⋅t⎤     ⎡1  -1  2   1 - 3⋅t⎤        ⎤
⎢⎢         ⎥     ⎢                  ⎥     ⎢                  ⎥        ⎥
⎢⎢2  3   -1⎥, 3, ⎢2  3   -1  -t - 3 ⎥, 3, ⎢0  5   -5  5⋅t - 5⎥, (3, 3)⎥
⎢⎢         ⎥     ⎢                  ⎥     ⎢                  ⎥        ⎥
⎣⎣1  1   1 ⎦     ⎣1  1   1     -t   ⎦     ⎣0  0   1      1   ⎦        ⎦
{(-2⋅t - 1, t, 1)}
{t: y, x: -2⋅y - 1, z: 1}


In [6]:
# p14.e2

sistemas = [
            [Eq(x-2*y+3*z,-3),Eq(x+y+z,0), Eq(2*x+5*y+9*z,3)],
    [Eq(-x-y-z,-1),Eq(5*x+y+2*z,-3), Eq(4*x+2*y+z,0)],
           ]

for sist in sistemas:
  pprint(sist)
  display_latex("$"+sistema_a_latex(sist)+"$",raw=True)
  display("$"+sistema_a_latex(sist)+"$")  
  A, b = linear_eq_to_matrix(sist,[x,y,z])
  AA = A.row_join(b)
  #pprint(linear_eq_to_matrix(sist,[x,y,z]))
  pprint([A,A.rank(),AA,AA.rank(),  AA.LUdecomposition()[1],A.shape])
  #pprint([A.LUdecomposition(),A.LUsolve(b)])
  pprint(linsolve(sist,[x,y,z]))
  pprint(solve(sist))

[x - 2⋅y + 3⋅z = -3, x + y + z = 0, 2⋅x + 5⋅y + 9⋅z = 3]


'$$\\left\\{ \\begin{matrix}x - 2 y + 3 z = -3 \\\\ x + y + z = 0 \\\\ 2 x + 5 y + 9 z = 3 \\\\ \\end{matrix}\\right.$$'

⎡⎡1  -2  3⎤     ⎡1  -2  3  -3⎤     ⎡1  -2  3   -3⎤        ⎤
⎢⎢        ⎥     ⎢            ⎥     ⎢             ⎥        ⎥
⎢⎢1  1   1⎥, 3, ⎢1  1   1  0 ⎥, 3, ⎢0  3   -2  3 ⎥, (3, 3)⎥
⎢⎢        ⎥     ⎢            ⎥     ⎢             ⎥        ⎥
⎣⎣2  5   9⎦     ⎣2  5   9  3 ⎦     ⎣0  0   9   0 ⎦        ⎦
{(-1, 1, 0)}
{x: -1, y: 1, z: 0}
[-x - y - z = -1, 5⋅x + y + 2⋅z = -3, 4⋅x + 2⋅y + z = 0]


'$$\\left\\{ \\begin{matrix}- x - y - z = -1 \\\\ 5 x + y + 2 z = -3 \\\\ 4 x + 2 y + z = 0 \\\\ \\end{matrix}\\right.$$'

⎡⎡-1  -1  -1⎤     ⎡-1  -1  -1  -1⎤     ⎡-1  -1   -1   -1⎤        ⎤
⎢⎢          ⎥     ⎢              ⎥     ⎢                ⎥        ⎥
⎢⎢5   1   2 ⎥, 3, ⎢5   1   2   -3⎥, 3, ⎢0   -4   -3   -8⎥, (3, 3)⎥
⎢⎢          ⎥     ⎢              ⎥     ⎢                ⎥        ⎥
⎣⎣4   2   1 ⎦     ⎣4   2   1   0 ⎦     ⎣0   0   -3/2  0 ⎦        ⎦
{(-1, 2, 0)}
{x: -1, y: 2, z: 0}


In [7]:
# p14.e2

sistemas = [[Eq(2*x-y+3*z,-3),Eq(x+y+z,0), Eq(2*x+5*y+z,3)],
            [Eq(-x-y-z,-1),Eq(5*x+y+2*z,-3), Eq(4*x+2*y+z,0)],
            [Eq(3*x+5*y-8*z,2),Eq(5*x+3*y-8*z,2), Eq(-8*x+5*y+3*z,2)],
           ]

for sist in sistemas:
  pprint(sist)
  display_latex("$"+sistema_a_latex(sist)+"$",raw=True)  
  A, b = linear_eq_to_matrix(sist,[x,y,z])
  AA = A.row_join(b)
  #pprint(linear_eq_to_matrix(sistema,[x,y,z]))
  pprint([A,A.rank(),AA,AA.rank(),  AA.LUdecomposition()[1] ,A.shape])
#   pprint([A.LUdecomposition(),A.LUsolve(b)])
  pprint(linsolve(sist,[x,y,z]))
  pprint(solve(sist))
  

[2⋅x - y + 3⋅z = -3, x + y + z = 0, 2⋅x + 5⋅y + z = 3]


⎡⎡2  -1  3⎤     ⎡2  -1  3  -3⎤     ⎡2  -1    3    -3 ⎤        ⎤
⎢⎢        ⎥     ⎢            ⎥     ⎢                 ⎥        ⎥
⎢⎢1  1   1⎥, 2, ⎢1  1   1  0 ⎥, 2, ⎢0  3/2  -1/2  3/2⎥, (3, 3)⎥
⎢⎢        ⎥     ⎢            ⎥     ⎢                 ⎥        ⎥
⎣⎣2  5   1⎦     ⎣2  5   1  3 ⎦     ⎣0   0    0     0 ⎦        ⎦
⎧⎛  4⋅z      z       ⎞⎫
⎨⎜- ─── - 1, ─ + 1, z⎟⎬
⎩⎝   3       3       ⎠⎭
⎧     4⋅z         z    ⎫
⎨x: - ─── - 1, y: ─ + 1⎬
⎩      3          3    ⎭
[-x - y - z = -1, 5⋅x + y + 2⋅z = -3, 4⋅x + 2⋅y + z = 0]


⎡⎡-1  -1  -1⎤     ⎡-1  -1  -1  -1⎤     ⎡-1  -1   -1   -1⎤        ⎤
⎢⎢          ⎥     ⎢              ⎥     ⎢                ⎥        ⎥
⎢⎢5   1   2 ⎥, 3, ⎢5   1   2   -3⎥, 3, ⎢0   -4   -3   -8⎥, (3, 3)⎥
⎢⎢          ⎥     ⎢              ⎥     ⎢                ⎥        ⎥
⎣⎣4   2   1 ⎦     ⎣4   2   1   0 ⎦     ⎣0   0   -3/2  0 ⎦        ⎦
{(-1, 2, 0)}
{x: -1, y: 2, z: 0}
[3⋅x + 5⋅y - 8⋅z = 2, 5⋅x + 3⋅y - 8⋅z = 2, -8⋅x + 5⋅y + 3⋅z = 2]


⎡⎡3   5  -8⎤     ⎡3   5  -8  2⎤     ⎡3    5     -8    2  ⎤        ⎤
⎢⎢         ⎥     ⎢            ⎥     ⎢                    ⎥        ⎥
⎢⎢5   3  -8⎥, 2, ⎢5   3  -8  2⎥, 3, ⎢0  -16/3  16/3  -4/3⎥, (3, 3)⎥
⎢⎢         ⎥     ⎢            ⎥     ⎢                    ⎥        ⎥
⎣⎣-8  5  3 ⎦     ⎣-8  5  3   2⎦     ⎣0    0     0    11/4⎦        ⎦
∅
[]


In [8]:
# p14.e3y4

sistemas = [[Eq(2*x+4*y+5*z,1),Eq(x+3*y+3*z,-1), Eq(3*x+3*y+2*z,2)],
            [Eq(x+y+z,2),Eq(x+2*y-3*z,8), Eq(2*x-2*y+2*z,-4)]
           ]

for sist in sistemas:
  pprint(sist)
  display_latex("$"+sistema_a_latex(sist)+"$",raw=True)   
  A, b = linear_eq_to_matrix(sist,[x,y,z])
  AA = A.row_join(b)
  #pprint(linear_eq_to_matrix(sistema,[x,y,z]))
  pprint([A,A.rank(), A.inv(),AA,AA.rank(),A.shape])
  pprint([A.LUdecomposition(),A.LUsolve(b)])
  pprint(linsolve(sist,[x,y,z]))
  pprint(solve(sist))
  pprint(A.inv()*b)
  pprint(solve_linear_system(AA,x,y,z))
  #cramer
  for i in range(A.shape[1]):
    AA.col_swap(i,3)
    pprint("Delta_i, A.det, s_i")
    pprint([AA[:,:-1].det(),A.det(),AA[:,:-1].det()/A.det()])
    AA.col_swap(i,3)
    #pprint(AA)


[2⋅x + 4⋅y + 5⋅z = 1, x + 3⋅y + 3⋅z = -1, 3⋅x + 3⋅y + 2⋅z = 2]


⎡⎡2  4  5⎤     ⎡3/8   -7/8  3/8 ⎤  ⎡2  4  5  1 ⎤           ⎤
⎢⎢       ⎥     ⎢                ⎥  ⎢           ⎥           ⎥
⎢⎢1  3  3⎥, 3, ⎢-7/8  11/8  1/8 ⎥, ⎢1  3  3  -1⎥, 3, (3, 3)⎥
⎢⎢       ⎥     ⎢                ⎥  ⎢           ⎥           ⎥
⎣⎣3  3  2⎦     ⎣3/4   -3/4  -1/4⎦  ⎣3  3  2  2 ⎦           ⎦
⎡⎛⎡ 1   0   0⎤  ⎡2  4   5 ⎤    ⎞  ⎡2 ⎤⎤
⎢⎜⎢          ⎥  ⎢         ⎥    ⎟  ⎢  ⎥⎥
⎢⎜⎢1/2  1   0⎥, ⎢0  1  1/2⎥, []⎟, ⎢-2⎥⎥
⎢⎜⎢          ⎥  ⎢         ⎥    ⎟  ⎢  ⎥⎥
⎣⎝⎣3/2  -3  1⎦  ⎣0  0  -4 ⎦    ⎠  ⎣1 ⎦⎦
{(2, -2, 1)}
{x: 2, y: -2, z: 1}
⎡2 ⎤
⎢  ⎥
⎢-2⎥
⎢  ⎥
⎣1 ⎦
{x: 2, y: -2, z: 1}
Delta_i, A.det, s_i
[-16, -8, 2]
Delta_i, A.det, s_i
[16, -8, -2]
Delta_i, A.det, s_i
[-8, -8, 1]
[x + y + z = 2, x + 2⋅y - 3⋅z = 8, 2⋅x - 2⋅y + 2⋅z = -4]


⎡⎡1  1   1 ⎤     ⎡1/8  1/4   5/16 ⎤  ⎡1  1   1   2 ⎤           ⎤
⎢⎢         ⎥     ⎢                ⎥  ⎢             ⎥           ⎥
⎢⎢1  2   -3⎥, 3, ⎢1/2   0    -1/4 ⎥, ⎢1  2   -3  8 ⎥, 3, (3, 3)⎥
⎢⎢         ⎥     ⎢                ⎥  ⎢             ⎥           ⎥
⎣⎣2  -2  2 ⎦     ⎣3/8  -1/4  -1/16⎦  ⎣2  -2  2   -4⎦           ⎦
⎡⎛⎡1  0   0⎤  ⎡1  1   1 ⎤    ⎞  ⎡1 ⎤⎤
⎢⎜⎢        ⎥  ⎢         ⎥    ⎟  ⎢  ⎥⎥
⎢⎜⎢1  1   0⎥, ⎢0  1  -4 ⎥, []⎟, ⎢2 ⎥⎥
⎢⎜⎢        ⎥  ⎢         ⎥    ⎟  ⎢  ⎥⎥
⎣⎝⎣2  -4  1⎦  ⎣0  0  -16⎦    ⎠  ⎣-1⎦⎦
{(1, 2, -1)}
{x: 1, y: 2, z: -1}
⎡1 ⎤
⎢  ⎥
⎢2 ⎥
⎢  ⎥
⎣-1⎦
{x: 1, y: 2, z: -1}
Delta_i, A.det, s_i
[-16, -16, 1]
Delta_i, A.det, s_i
[-32, -16, 2]
Delta_i, A.det, s_i
[16, -16, -1]


In [9]:
# p14.e1bis

sistemas = [[Eq(2*x-y+z,1),Eq(x-2*y+3*z,4), Eq(x+y+3*z,0)],
            [Eq(5*x-2*y+z,-1),Eq(-2*x+y-3*z,4), Eq(3*x-y-2*z,0)],
            [Eq(3*x+2*y-z,1),Eq(-2*x+3*y-2*z,-1),Eq(x+5*y-3*x,0)],
            [Eq(2*x-3*y+z,0),Eq(x+2*y-z,0),Eq(4*x+y-z,0)]
           ]

for sist in sistemas:
  pprint(sist)
  display_latex("$"+sistema_a_latex(sist)+"$",raw=True)   
  A, b = linear_eq_to_matrix(sist,[x,y,z])
  AA = A.row_join(b)
  pprint([A,A.rank(),AA,AA.rank(),A.shape])

[2⋅x - y + z = 1, x - 2⋅y + 3⋅z = 4, x + y + 3⋅z = 0]


⎡⎡2  -1  1⎤     ⎡2  -1  1  1⎤           ⎤
⎢⎢        ⎥     ⎢           ⎥           ⎥
⎢⎢1  -2  3⎥, 3, ⎢1  -2  3  4⎥, 3, (3, 3)⎥
⎢⎢        ⎥     ⎢           ⎥           ⎥
⎣⎣1  1   3⎦     ⎣1  1   3  0⎦           ⎦
[5⋅x - 2⋅y + z = -1, -2⋅x + y - 3⋅z = 4, 3⋅x - y - 2⋅z = 0]


⎡⎡5   -2  1 ⎤     ⎡5   -2  1   -1⎤           ⎤
⎢⎢          ⎥     ⎢              ⎥           ⎥
⎢⎢-2  1   -3⎥, 2, ⎢-2  1   -3  4 ⎥, 3, (3, 3)⎥
⎢⎢          ⎥     ⎢              ⎥           ⎥
⎣⎣3   -1  -2⎦     ⎣3   -1  -2  0 ⎦           ⎦
[3⋅x + 2⋅y - z = 1, -2⋅x + 3⋅y - 2⋅z = -1, -2⋅x + 5⋅y = 0]


⎡⎡3   2  -1⎤     ⎡3   2  -1  1 ⎤           ⎤
⎢⎢         ⎥     ⎢             ⎥           ⎥
⎢⎢-2  3  -2⎥, 3, ⎢-2  3  -2  -1⎥, 3, (3, 3)⎥
⎢⎢         ⎥     ⎢             ⎥           ⎥
⎣⎣-2  5  0 ⎦     ⎣-2  5  0   0 ⎦           ⎦
[2⋅x - 3⋅y + z = 0, x + 2⋅y - z = 0, 4⋅x + y - z = 0]


⎡⎡2  -3  1 ⎤     ⎡2  -3  1   0⎤           ⎤
⎢⎢         ⎥     ⎢            ⎥           ⎥
⎢⎢1  2   -1⎥, 2, ⎢1  2   -1  0⎥, 2, (3, 3)⎥
⎢⎢         ⎥     ⎢            ⎥           ⎥
⎣⎣4  1   -1⎦     ⎣4  1   -1  0⎦           ⎦


In [10]:
# p14.e2bis

sistemas = [[Eq(x+2*y+z,0),Eq(2*x+y+2*z,1), Eq(5*x+7*y+5*z,1)],
            [Eq(3*x+2*y+z,4),Eq(-2*x+3*y+2*z,4), Eq(x+5*y+3*z,1),Eq(-4*x+6*y+4*z,-3)],
            [Eq(x+2*y+3*z,2),Eq(3*x+2*y+2*z,2),Eq(x-y+4*z,1),Eq(2*x+y-z,3)],
           ]

for sist in sistemas:
  pprint(sist)
  display_latex("$"+sistema_a_latex(sist)+"$",raw=True)   
  A, b = linear_eq_to_matrix(sist,[x,y,z])
  AA = A.row_join(b)
  pprint([A,A.rank(),AA,AA.rank(),A.shape, AA.LUdecomposition()[1]])
  pprint(linsolve(sist,[x,y,z]))  
#   if A.rank() == AA.rank() :
#         pprint(linsolve(sist,[x,y,z]))

[x + 2⋅y + z = 0, 2⋅x + y + 2⋅z = 1, 5⋅x + 7⋅y + 5⋅z = 1]


⎡⎡1  2  1⎤     ⎡1  2  1  0⎤             ⎡1  2   1  0⎤⎤
⎢⎢       ⎥     ⎢          ⎥             ⎢           ⎥⎥
⎢⎢2  1  2⎥, 2, ⎢2  1  2  1⎥, 2, (3, 3), ⎢0  -3  0  1⎥⎥
⎢⎢       ⎥     ⎢          ⎥             ⎢           ⎥⎥
⎣⎣5  7  5⎦     ⎣5  7  5  1⎦             ⎣0  0   0  0⎦⎦
{(2/3 - z, -1/3, z)}
[3⋅x + 2⋅y + z = 4, -2⋅x + 3⋅y + 2⋅z = 4, x + 5⋅y + 3⋅z = 1, -4⋅x + 6⋅y + 4⋅z 
= -3]


⎡⎡3   2  1⎤     ⎡3   2  1  4 ⎤             ⎡3   2     1    4  ⎤⎤
⎢⎢        ⎥     ⎢            ⎥             ⎢                  ⎥⎥
⎢⎢-2  3  2⎥     ⎢-2  3  2  4 ⎥             ⎢0  13/3  8/3  20/3⎥⎥
⎢⎢        ⎥, 2, ⎢            ⎥, 3, (4, 3), ⎢                  ⎥⎥
⎢⎢1   5  3⎥     ⎢1   5  3  1 ⎥             ⎢0   0     0    -7 ⎥⎥
⎢⎢        ⎥     ⎢            ⎥             ⎢                  ⎥⎥
⎣⎣-4  6  4⎦     ⎣-4  6  4  -3⎦             ⎣0   0     0    0  ⎦⎦
∅
[x + 2⋅y + 3⋅z = 2, 3⋅x + 2⋅y + 2⋅z = 2, x - y + 4⋅z = 1, 2⋅x + y - z = 3]


⎡                                           ⎡1  2    3    2 ⎤⎤
⎢⎡1  2   3 ⎤     ⎡1  2   3   2⎤             ⎢               ⎥⎥
⎢⎢         ⎥     ⎢            ⎥             ⎢0  -4   -7   -4⎥⎥
⎢⎢3  2   2 ⎥     ⎢3  2   2   2⎥             ⎢               ⎥⎥
⎢⎢         ⎥, 3, ⎢            ⎥, 4, (4, 3), ⎢0  0   25/4  2 ⎥⎥
⎢⎢1  -1  4 ⎥     ⎢1  -1  4   1⎥             ⎢               ⎥⎥
⎢⎢         ⎥     ⎢            ⎥             ⎢             64⎥⎥
⎢⎣2  1   -1⎦     ⎣2  1   -1  3⎦             ⎢0  0    0    ──⎥⎥
⎣                                           ⎣             25⎦⎦
∅


In [11]:
# p14.e3bis

sistemas = [[Eq(y+z,1),Eq(x+y-3*z,8), Eq(x-2*y,-1)],
            [Eq(2*x+3*y+z,1),Eq(2*x+2*y+z,1), Eq(4*x+5*y+2*z,2)]
           ]

for sist in sistemas:
  pprint(sist)
  display_latex("$"+sistema_a_latex(sist)+"$",raw=True)   
  A, b = linear_eq_to_matrix(sist,[x,y,z])
  AA = A.row_join(b)
  #pprint(linear_eq_to_matrix(sistema,[x,y,z]))
  pprint([A,A.rank(),AA,AA.rank(),A.shape,AA.LUdecomposition()[1],A.det()])
  pprint(linsolve(sist,[x,y,z]))
#   pprint(solve(sist))
#   pprint(A.inv()*b)
#   pprint(solve_linear_system(AA,x,y,z))
#   #cramer
#   for i in range(A.shape[1]):
#     AA.col_swap(i,3)
#     pprint("Delta_i, A.det, s_i")
#     pprint([AA[:,:-1].det(),A.det(),AA[:,:-1].det()/A.det()])
#     AA.col_swap(i,3)
#     #pprint(AA)


[y + z = 1, x + y - 3⋅z = 8, x - 2⋅y = -1]


⎡⎡0  1   1 ⎤     ⎡0  1   1   1 ⎤             ⎡1  1  -3  8 ⎤    ⎤
⎢⎢         ⎥     ⎢             ⎥             ⎢            ⎥    ⎥
⎢⎢1  1   -3⎥, 3, ⎢1  1   -3  8 ⎥, 3, (3, 3), ⎢0  1  1   1 ⎥, -6⎥
⎢⎢         ⎥     ⎢             ⎥             ⎢            ⎥    ⎥
⎣⎣1  -2  0 ⎦     ⎣1  -2  0   -1⎦             ⎣0  0  6   -6⎦    ⎦
{(3, 2, -1)}
[2⋅x + 3⋅y + z = 1, 2⋅x + 2⋅y + z = 1, 4⋅x + 5⋅y + 2⋅z = 2]


⎡⎡2  3  1⎤     ⎡2  3  1  1⎤             ⎡2  3   1  1⎤   ⎤
⎢⎢       ⎥     ⎢          ⎥             ⎢           ⎥   ⎥
⎢⎢2  2  1⎥, 2, ⎢2  2  1  1⎥, 2, (3, 3), ⎢0  -1  0  0⎥, 0⎥
⎢⎢       ⎥     ⎢          ⎥             ⎢           ⎥   ⎥
⎣⎣4  5  2⎦     ⎣4  5  2  2⎦             ⎣0  0   0  0⎦   ⎦
⎧⎛1   z      ⎞⎫
⎨⎜─ - ─, 0, z⎟⎬
⎩⎝2   2      ⎠⎭


In [12]:
# p14.e4bis

sistemas = [
#         [[Eq(-3*x+2*y+3*z,-2),Eq(2*x+k*y-5*z,-4),Eq(x+y+2*z,2)],k],
#         [[Eq(x + y + z , k + 2),Eq(x - k*y + z , 1),Eq(k*x + y + z , 4)],k],
#         [[Eq(x-2*y+z,3),Eq(y+2*z,0),Eq(3*y+7*z,m)],m],


#         [[Eq(k*x+(k**2+1)*y+k*z,k),Eq(x+k*y+z,0),Eq(x+(k+1)*y+k**2*z,2*k-1)],k],
#     [[Eq(m*x + m*z + y*(m**2 + 1), m), Eq(m*y + x + z, 0), Eq(m**2*z + x + y*(m + 1), 2*m - 1)],m],
#            [[Eq(k*x + y + z , k),Eq(x + k*y + z , k), Eq(x + y + k*z , k)],k],
    #     [[Eq(x-2*y+3*z,-3),Eq(2*x+y,2), Eq(-x-8*y+9*z,3)],h],
    
    # Ejercicio de examen
#     [[Eq(x+y+z,-2),Eq(-k*x+3*y+z,-7),Eq(x+2*y+(k+2)*z,-5)],k],
    [[Eq(k*x+k*y+k**2*z,12),Eq(x-y+k*z,6),Eq(2*k*x-2*y+2*z,8)],k],

    
]

for s in sistemas :
    sist, par = s
    print("===============================================================")
    sistema_parametro(sist,param=par)



Discusión y resolución del sistema:


-----------------------------------------------------------


**Resolución por Gauss**
- - - - - - - - - - - -


**Método de la matríz inversa**
- - - - - - - - - - - - - - - 


**Por Cramer**
- - - - - - - -


**Resolución por Gauss**
- - - - - - - - - - - -


**Método de la matríz inversa**
- - - - - - - - - - - - - - - 


**Por Cramer**
- - - - - - - -


TypeError: bad operand type for unary -: 'BooleanFalse'

In [ ]:
sistema_solucion([Eq(k*x+k*y+k**2*z,12).subs(k,2),Eq(x-y+k*z,6).subs(k,2),Eq(2*k*x-2*y+2*z,8).subs(k,2)],[x,y,z])

In [13]:
# p14.e5

sistemas = [
            [[Eq(2*x - 5*y + 3*z , 0),Eq(x - y + z , 0),Eq(3*x + m*y + z , 0)],m],
            [[Eq(x + m*y - z , 0),Eq(12*x - 3*y - 2*z , 0),Eq(x - 2*y + z , 0)],m],
            [[Eq(x + y + 2*z , 0),Eq(m*x + y - z , m - 2), Eq(3*x + m*y + z , m - 2)],m],
                [[Eq(x -2*y + z , 1),Eq(m*x + y - z , 1), Eq(3*x + 4*y - 2*z , -3 )],m],
            [[Eq(2*x - 3*y + z , 0),Eq(x - k*y - 3*z , 0), Eq(5*x + 2*y - z , 0)],k],

#         [[Eq(k*x+(k**2+1)*y+k*z,k),Eq(x+k*y+z,0),Eq(x+(k+1)*y+k**2*z,2*k-1)],k],
#     [[Eq(m*x + m*z + y*(m**2 + 1), m), Eq(m*y + x + z, 0), Eq(m**2*z + x + y*(m + 1), 2*m - 1)],m],
#            [[Eq(k*x + y + z , k),Eq(x + k*y + z , k), Eq(x + y + k*z , k)],k],
    #     [[Eq(x-2*y+3*z,-3),Eq(2*x+y,2), Eq(-x-8*y+9*z,3)],h],
]

for s in sistemas :
    sist, par = s
    print("===============================================================")
    sistema_parametro(sist,param=par)



Discusión y resolución del sistema:


-----------------------------------------------------------


**Resolución por Gauss**
- - - - - - - - - - - -


**Método de la matríz inversa**
- - - - - - - - - - - - - - - 


**Por Cramer**
- - - - - - - -


...

-----------------------------------------------------------


**Resolución por Gauss**
- - - - - - - - - - - -


**Método de la matríz inversa**
- - - - - - - - - - - - - - - 


**Por Cramer**
- - - - - - - -


**Resolución por Gauss**
- - - - - - - - - - - -


In [ ]:
# Antiguo

# p14.e5

sistemas = [
            [Eq(-3*x+2*y+3*z,-2),Eq(2*x+k*y-5*z,-4),Eq(x+y+2*z,2)],
           [Eq(k*x + y + z , k),Eq(x + k*y + z , k), Eq(x + y + k*z , k)],
            [Eq(x + y + z , k + 2),Eq(x - k*y + z , 1),Eq(k*x + y + z , 4)],
            [Eq(k*x+(k**2+1)*y+k*z,k),Eq(x+k*y+z,0),Eq(x+(k+1)*y+k**2*z,2*k-1)],
            [Eq(2*x - 5*y + 3*z , 0),Eq(x - y + z , 0),Eq(3*x + k*y + z , 0)],
            [Eq(x + k*y - z , 0),Eq(12*x - 3*y - 2*z , 0),Eq(x - 2*y + z , 0)],
            [Eq(x + y + 2*z , 0),Eq(k*x + y - z , k - 2), Eq(3*x + k*y + z , k - 2)],
            [Eq(k*x+y,1),Eq(2*x+(k+1)*y,2),Eq(x+k*y,1)], 
            [Eq(y+k*z,1),Eq(k*x-y+z,1),Eq(y-z,-k)]
            ]

for sist in sistemas :
    display_latex("$"+sistema_a_latex(sist)+"$",raw=True)
    print(discusion_solucion(sist)['solucion_latex'])



In [15]:
# p14.e6a

sistemas = [[r"""6. Una compañía aérea realiza vuelos a tres ciudades, A, B y C. Calcula el precio de los billetes a cada ciudad con
la siguiente información. Si vende 10 billetes para ir a la ciudad A, 5 para B y ninguno para C, ingresa 925 euros;
si vende 12 billetes para A, 8 para B y ninguno para C, ingresa 1200 euros; y si vende 6 billetes para A, 5 para B y
8 para C, ingresa 861 euros.""",[Eq(10*x+5*y,925),Eq(12*x+8*y,1200), Eq(6*x+5*y+8*z,861)]],
            [r"""7. En una competición deportiva participan 50 atletas distribuidos, según la edad, en tres categorías: Infantiles,
Cadetes y Juveniles. El doble del número de atletas infantiles excede en una unidad al número de atletas cadetes
y, además, coincide con el quíntuplo del número de atletas juveniles. Determina el número de atletas de cada
categoría.""",[Eq(2*x,y+1),Eq(2*x,5*z), Eq(x+y+z,50)]], 
            #[r"""8. La suma de las tres cifras de un número es ocho. Sabiendo que la cifra de las centenas es igual que la de lasdecenas, y que ambas suman la cifra de las unidades, averigua dicho número.""",[Eq(x+y+z,8),Eq(x,y), Eq(x+y=z)]],
[r"""8. La suma de las tres cifras de un número es ocho. Sabiendo que la cifra de las centenas 
es igual que la de las decenas, y que ambas suman la cifra de las unidades, 
averigua dicho número.""",[Eq(x+y+z,8),Eq(x,y), Eq(x+y,z)]],  
            [r"""9. Un alumno de 1.° de Bachillerato emplea en la compra de tres lápices, un sacapuntas y dos gomas de borrar,
tres euros. El doble del precio de un lápiz excede en cinco céntimos de euro a la suma de los precios de un
sacapuntas y una goma de borrar. Si cada lápiz costara cinco céntimos de euro más, entonces su precio duplicaría
al de una goma de borrar. Determina el precio de un lápiz, de un sacapuntas y de una goma de borrar.""",
            [Eq(3*x+y+2*z,300),Eq(2*x,y+z+5),Eq(x+5,2*z)]],
            [r"""10. Para la compra de un artículo de precio 10,70 euros se utilizan monedas de 1 euro, de 50 céntimos de euro y
de 20 céntimos de euro. El número total de monedas excede en una unidad al triple de monedas de 1 euro. El 30
% de la suma del número de monedas de 1 euro con el doble del número de monedas de 50 céntimos coincide
con el número de monedas de 20 céntimos. Determina el número de monedas que se utilizan de cada clase.""",
            [Eq(100*x+50*y+20*z,1070),Eq(x+y+z,3*x+1),Eq(30/100*(x+2*y),z)]],
            [r"""11. En un grupo de 2.° de Bachillerato los alumnos tienen como materia optativa una de estas tres asignaturas:
Literatura, Psicología o Francés. El número de alumnos matriculados en Literatura representa el 60% del total de
alumnos del grupo. Si tres alumnos de Psicología se hubiesen matriculado en Francés, entonces estas dos
asignaturas tendrían el mismo número de alumnos. Finalmente, el doble de la diferencia del número de alumnos
matriculados en Literatura y en Psicología es el triple de la diferencia de los matriculados en Psicología y en
Francés. Calcula el número de alumnos matriculados en cada una de las materias optativas y el número de
alumnos del grupo.""",
            [Eq(x,60/100*(x+y+z)),Eq(y-3,z+3),Eq(2*(x-y),3*(y-z))]],
            [r"""12. Si la altura de Sergio aumentase el triple de la diferencia entre las alturas de Martín y Jesús, Sergio sería igual
de alto que Jesús. Las alturas de los tres suman 515 cm. Ocho veces la altura de Martín es lo mismo que nueve
veces la de Sergio. Halla la estatura de cada uno.""",
            [Eq(x+3*(y-z),z),Eq(x+y+z,515),Eq(8*y,9*x)]],
            [r"""13. En un instituto se imparten enseñanzas de ESO, Bachillerato y Ciclos Formativos. La suma del número de
los alumnos de Bachillerato y del doble de los alumnos de Ciclos Formativos excede en 100 al número de los
alumnos de ESO. Si sumamos el 40% de los alumnos matriculados en ESO con el 30% de los matriculados en
Bachillerato y con el 20% de los matriculados en Ciclos Formativos, se obtiene un número que excede en 45
unidades al 30% del número total de alumnos. Sabiendo que cursan estos tres tipos de enseñanza un total de 1
200 alumnos, halla el número de alumnos matriculados en cada tipo de enseñanza.""",
            [Eq(y+2*z,x+100),Eq(40*x+30*y+20*z,30*(x+y+z)+4500),Eq(x+y+z,1200)]],
            [r"""14. Una persona deja en herencia a sus hijos todo su dinero con las siguientes condiciones: al mayor le deja la
media de lo que le deja a los otros más 30 000 euros; al mediano, exactamente la media del dinero de los otros; y
al pequeño, la media del dinero de los otros menos 30 000 euros. De esta forma, ¿pueden los hijos saber cuánto
dinero ha heredado cada uno?""",
            [Eq(x,y/2+z/2+30000),Eq(y,x/2+z/2),Eq(z,y/2+x/2-30000)]],            
            [r"""15. A Victoria le van a confeccionar su vestido de novia, por el que tendrá que pagar 1 742 euros. En la confección
se utilizarán 15 metros de tela entre raso, seda y pasamanería bordada. Los metros de raso serán el doble que los
de seda. Los precios del raso, de la seda y la pasamanería son 55,62 y 68 euros el metro, respectivamente. Si del
precio total se destinan 500 euros a la mano de obra y 350 euros para beneficio del comerciante, ¿cuántos metros
de cada material serán necesarios?""",
            [Eq(x+y+z,15),Eq(x,2*y),Eq(55*x+62*y+68*z,1742-500-350)]],
            [r"""16. Lucía ha realizado tres pruebas escritas de una asignatura y la nota media de las tres ha sido 8,5 puntos. Si la
nota media de las dos primeras es 8 y la nota media de las dos últimas es 9, halla la calificación de cada una de las
pruebas.""",
            [Eq((x+y+z)/3,8.5),Eq((x+y)/2,8),Eq((y+z)/2,9)]],
           ]

for sist in sistemas:
    print("----------------------------------------------")
    print(sist[0])
    print("----------------------------------------------")

    display_latex(sistema_a_latex(sist[1]),raw=True)
    sistema_solucion(sist[1], lista_incognitas(sist[1]))
    


----------------------------------------------
6. Una compañía aérea realiza vuelos a tres ciudades, A, B y C. Calcula el precio de los billetes a cada ciudad con
la siguiente información. Si vende 10 billetes para ir a la ciudad A, 5 para B y ninguno para C, ingresa 925 euros;
si vende 12 billetes para A, 8 para B y ninguno para C, ingresa 1200 euros; y si vende 6 billetes para A, 5 para B y
8 para C, ingresa 861 euros.
----------------------------------------------


**Resolución por Gauss**
- - - - - - - - - - - -


**Método de la matríz inversa**
- - - - - - - - - - - - - - - 


**Por Cramer**
- - - - - - - -


----------------------------------------------
7. En una competición deportiva participan 50 atletas distribuidos, según la edad, en tres categorías: Infantiles,
Cadetes y Juveniles. El doble del número de atletas infantiles excede en una unidad al número de atletas cadetes
y, además, coincide con el quíntuplo del número de atletas juveniles. Determina el número de atletas de cada
categoría.
----------------------------------------------


...

**Por Cramer**
- - - - - - - -


----------------------------------------------
16. Lucía ha realizado tres pruebas escritas de una asignatura y la nota media de las tres ha sido 8,5 puntos. Si la
nota media de las dos primeras es 8 y la nota media de las dos últimas es 9, halla la calificación de cada una de las
pruebas.
----------------------------------------------


**Resolución por Gauss**
- - - - - - - - - - - -


**Método de la matríz inversa**
- - - - - - - - - - - - - - - 


**Por Cramer**
- - - - - - - -


In [21]:
# exámenes

sistemas = [[r"""Un hipermercado inicia una campaña de ofertas. En la primera de
ellas descuenta un 4% en un cierto producto A, un 6% en el producto B y un 5% en el producto
C. A las dos semanas pone en marcha la segunda oferta descontando un 8% sobre el precio inicial
de A, un 10% sobre el precio inicial de B y un 6% sobre el precio inicial de C.
Se sabe que si un cliente compra durante la primera oferta un producto A, dos B y tres C, se
ahorra 16 euros respecto del precio inicial. Si compra tres productos A, uno B y cinco C en la
segunda oferta, el ahorro es de 29 euros. Si compra un producto A, uno B y uno C, sin ningún
tipo de descuento, debe abonar 135 euros.
Calcúlese el precio de cada producto antes de las ofertas""",[Eq(x+y+z,135),Eq(4*x+2*6*y+3*5*z,1600), 
                                                              Eq(3*8*x+10*y+5*6*z,2900)]],
            [r"""Una empresa desea disponer de dinero en efectivo en euros, dólares
y libras esterlinas. El valor total entre las tres monedas ha de ser igual a 264000 euros. Se quiere
que el valor del dinero disponible en euros sea el doble del valor del dinero en dólares, y que el
valor del dinero en libras esterlinas sea la décima parte del dinero en euros.
Si se supone que una libra esterlina es igual a 1,5 euros y un dólar es igual a 1,1 euros, se pide
determinar la cantidad de euros, dólares y libras esterlinas que la empresa ha de tener disponible.""",
            [Eq(10*x+11*y+15*z,2640000),Eq(10*x-22*y,0),Eq(15*z-x,0)]]
           ]

for sist in sistemas:
    print("----------------------------------------------")
    print(sist[0])
    print("----------------------------------------------")

    display_latex(sistema_a_latex(sist[1]),raw=True)
    sistema_solucion(sist[1], lista_incognitas(sist[1]))
    


----------------------------------------------
Un hipermercado inicia una campaña de ofertas. En la primera de
ellas descuenta un 4% en un cierto producto A, un 6% en el producto B y un 5% en el producto
C. A las dos semanas pone en marcha la segunda oferta descontando un 8% sobre el precio inicial
de A, un 10% sobre el precio inicial de B y un 6% sobre el precio inicial de C.
Se sabe que si un cliente compra durante la primera oferta un producto A, dos B y tres C, se
ahorra 16 euros respecto del precio inicial. Si compra tres productos A, uno B y cinco C en la
segunda oferta, el ahorro es de 29 euros. Si compra un producto A, uno B y uno C, sin ningún
tipo de descuento, debe abonar 135 euros.
Calcúlese el precio de cada producto antes de las ofertas
----------------------------------------------


**Resolución por Gauss**
- - - - - - - - - - - -


**Método de la matríz inversa**
- - - - - - - - - - - - - - - 


**Por Cramer**
- - - - - - - -


----------------------------------------------
Una empresa desea disponer de dinero en efectivo en euros, dólares
y libras esterlinas. El valor total entre las tres monedas ha de ser igual a 264000 euros. Se quiere
que el valor del dinero disponible en euros sea el doble del valor del dinero en dólares, y que el
valor del dinero en libras esterlinas sea la décima parte del dinero en euros.
Si se supone que una libra esterlina es igual a 1,5 euros y un dólar es igual a 1,1 euros, se pide
determinar la cantidad de euros, dólares y libras esterlinas que la empresa ha de tener disponible.
----------------------------------------------


**Resolución por Gauss**
- - - - - - - - - - - -


**Método de la matríz inversa**
- - - - - - - - - - - - - - - 


**Por Cramer**
- - - - - - - -


In [ ]:
# Junio 2001

sistemas = [[r"""Junio 2001""",[Eq(200*x+100*y+150*z,8600),Eq(x-2*z,0), Eq(x-3*y+3*z,0)]],
           ]

for sist in sistemas:
    print("----------------------------------------------")
    print(sist[0])
    print("----------------------------------------------")

    display_latex(sistema_a_latex(sist[1]),raw=True)
    sistema_solucion(sist[1], lista_incognitas(sist[1]))
    


In [ ]:
# Antiguo

# p14.e6a

sistemas = [[r"""6. Una compañía aérea realiza vuelos a tres ciudades, A, B y C. Calcula el precio de los billetes a cada ciudad con
la siguiente información. Si vende 10 billetes para ir a la ciudad A, 5 para B y ninguno para C, ingresa 925 euros;
si vende 12 billetes para A, 8 para B y ninguno para C, ingresa 1200 euros; y si vende 6 billetes para A, 5 para B y
8 para C, ingresa 861 euros.""",[Eq(10*x+5*y,925),Eq(12*x+8*y,1200), Eq(6*x+5*y+8*z,861)]],
            [r"""7. En una competición deportiva participan 50 atletas distribuidos, según la edad, en tres categorías: Infantiles,
Cadetes y Juveniles. El doble del número de atletas infantiles excede en una unidad al número de atletas cadetes
y, además, coincide con el quíntuplo del número de atletas juveniles. Determina el número de atletas de cada
categoría.""",[Eq(2*x,y+1),Eq(2*x,5*z), Eq(x+y+z,50)]], 
            #[r"""8. La suma de las tres cifras de un número es ocho. Sabiendo que la cifra de las centenas es igual que la de lasdecenas, y que ambas suman la cifra de las unidades, averigua dicho número.""",[Eq(x+y+z,8),Eq(x,y), Eq(x+y=z)]],
[r"""8. La suma de las tres cifras de un número es ocho. Sabiendo que la cifra de las centenas 
es igual que la de las decenas, y que ambas suman la cifra de las unidades, 
averigua dicho número.""",[Eq(x+y+z,8),Eq(x,y), Eq(x+y,z)]],  
            [r"""9. Un alumno de 1.° de Bachillerato emplea en la compra de tres lápices, un sacapuntas y dos gomas de borrar,
tres euros. El doble del precio de un lápiz excede en cinco céntimos de euro a la suma de los precios de un
sacapuntas y una goma de borrar. Si cada lápiz costara cinco céntimos de euro más, entonces su precio duplicaría
al de una goma de borrar. Determina el precio de un lápiz, de un sacapuntas y de una goma de borrar.""",
            [Eq(3*x+y+2*z,300),Eq(2*x,y+z+5),Eq(x+5,2*z)]],
            [r"""10. Para la compra de un artículo de precio 10,70 euros se utilizan monedas de 1 euro, de 50 céntimos de euro y
de 20 céntimos de euro. El número total de monedas excede en una unidad al triple de monedas de 1 euro. El 30
% de la suma del número de monedas de 1 euro con el doble del número de monedas de 50 céntimos coincide
con el número de monedas de 20 céntimos. Determina el número de monedas que se utilizan de cada clase.""",
            [Eq(100*x+50*y+20*z,1070),Eq(x+y+z,3*x+1),Eq(30/100*(x+2*y),z)]],
            [r"""11. En un grupo de 2.° de Bachillerato los alumnos tienen como materia optativa una de estas tres asignaturas:
Literatura, Psicología o Francés. El número de alumnos matriculados en Literatura representa el 60% del total de
alumnos del grupo. Si tres alumnos de Psicología se hubiesen matriculado en Francés, entonces estas dos
asignaturas tendrían el mismo número de alumnos. Finalmente, el doble de la diferencia del número de alumnos
matriculados en Literatura y en Psicología es el triple de la diferencia de los matriculados en Psicología y en
Francés. Calcula el número de alumnos matriculados en cada una de las materias optativas y el número de
alumnos del grupo.""",
            [Eq(x,60/100*(x+y+z)),Eq(y-3,z+3),Eq(2*(x-y),3*(y-z))]],
            [r"""12. Si la altura de Sergio aumentase el triple de la diferencia entre las alturas de Martín y Jesús, Sergio sería igual
de alto que Jesús. Las alturas de los tres suman 515 cm. Ocho veces la altura de Martín es lo mismo que nueve
veces la de Sergio. Halla la estatura de cada uno.""",
            [Eq(x+3*(y-z),z),Eq(x+y+z,515),Eq(8*y,9*x)]],
            [r"""13. En un instituto se imparten enseñanzas de ESO, Bachillerato y Ciclos Formativos. La suma del número de
los alumnos de Bachillerato y del doble de los alumnos de Ciclos Formativos excede en 100 al número de los
alumnos de ESO. Si sumamos el 40% de los alumnos matriculados en ESO con el 30% de los matriculados en
Bachillerato y con el 20% de los matriculados en Ciclos Formativos, se obtiene un número que excede en 45
unidades al 30% del número total de alumnos. Sabiendo que cursan estos tres tipos de enseñanza un total de 1
200 alumnos, halla el número de alumnos matriculados en cada tipo de enseñanza.""",
            [Eq(y+2*z,x+100),Eq(40*x+30*y+20*z,30*(x+y+z)+4500),Eq(x+y+z,1200)]],
            [r"""14. Una persona deja en herencia a sus hijos todo su dinero con las siguientes condiciones: al mayor le deja la
media de lo que le deja a los otros más 30 000 euros; al mediano, exactamente la media del dinero de los otros; y
al pequeño, la media del dinero de los otros menos 30 000 euros. De esta forma, ¿pueden los hijos saber cuánto
dinero ha heredado cada uno?""",
            [Eq(x,y/2+z/2+30000),Eq(y,x/2+z/2),Eq(z,y/2+x/2-30000)]],            
            [r"""15. A Victoria le van a confeccionar su vestido de novia, por el que tendrá que pagar 1 742 euros. En la confección
se utilizarán 15 metros de tela entre raso, seda y pasamanería bordada. Los metros de raso serán el doble que los
de seda. Los precios del raso, de la seda y la pasamanería son 55,62 y 68 euros el metro, respectivamente. Si del
precio total se destinan 500 euros a la mano de obra y 350 euros para beneficio del comerciante, ¿cuántos metros
de cada material serán necesarios?""",
            [Eq(x+y+z,15),Eq(x,2*y),Eq(55*x+62*y+68*z,1742-500-350)]],
            [r"""16. Lucía ha realizado tres pruebas escritas de una asignatura y la nota media de las tres ha sido 8,5 puntos. Si la
nota media de las dos primeras es 8 y la nota media de las dos últimas es 9, halla la calificación de cada una de las
pruebas.""",
            [Eq((x+y+z)/3,8.5),Eq((x+y)/2,8),Eq((y+z)/2,9)]],
           ]

for sist in sistemas:
  
  print(latex(sist[0]))
  pprint("Sistema:")
  pprint(sist[1])
  
  A, b = linear_eq_to_matrix(sist[1],[x,y,z])
  AA = A.row_join(b)


  #pprint(linear_eq_to_matrix(sistema,[x,y,z]))
  #print(latex(sist[1]))
  
  #pprint([A,A.rank(),AA,AA.rank(),A.shape])
  #pprint([A.LUdecomposition(),A.LUsolve(b)])
  #pprint(linsolve(sist[1],[x,y,z]))
  #pprint(solve(sist[1]))
  #pprint(A.inv()*b)
  pprint("Solución:")
  pprint(solve_linear_system(AA,x,y,z))
  
  #cramer
  pprint("Datos que salen en Cramer:")
  for i in range(A.shape[1]):
    AA.col_swap(i,3)
    pprint("Delta_i, A.det, solucion_i : ")
    pprint([AA[:,:-1].det(),A.det(),AA[:,:-1].det()/A.det()])
    AA.col_swap(i,3)
    #pprint(AA)
